<h1>MODEL BUILDER</h1>
<p>The purpose of this notebook is to build the proposed model, the dual fused EfficientNet Entry-Block (ENEB) with a Modified Residual Skip Block (MReSBlock)</p>

In [1]:
#LOAD DEPENDENCIES
import os
import logging
import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras import applications, Model, layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Layer, Conv2D, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Input, DepthwiseConv2D, Activation, Add, BatchNormalization

from tensorflow.keras.applications.efficientnet import EfficientNetB0 as selected_model, preprocess_input

model_architecture = "proposed_model"

activation_setting = tf.keras.activations.selu

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

<h2>Prepare essentials</h2>

In [2]:
#Custom Functions
data_path = "dataset"
model_kind = "proposed_model"
model_path = "models/" + model_kind + '/'

#Save Model Function
def save_m(directory, model):
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save(directory + '/' + model_architecture + '.h5')
    print("model saved")

In [3]:
#LOAD THE DATA

img_rows, img_cols = 224, 224
input_shape = (img_rows,img_cols,3)
model_input = Input(shape=input_shape)

print("The Input size is set to ", model_input) 

The Input size is set to  KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


<h2>ENEB Builder</h2>

In [4]:
# Builder 

#TRANSFER LEARNING
def model_builder_a(model_input):
    model_builder_a = selected_model(weights='imagenet', include_top=False, input_tensor=model_input)
    
#FINE TUNING
    for layer in model_builder_a.layers:
        layer.trainable = True
    for BatchNormalization in model_builder_a.layers:
        BatchNormalization.trainable = True
        
    for layer in model_builder_a.layers:
        layer._name = layer.name + '-a'
        
    x = model_builder_a.layers[-192].output

    model_a = Model(inputs=model_builder_a.input, outputs=x, name=model_architecture + '-a')
    return model_a

#INITIALIZE THE MODEL
model_a = model_builder_a(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
model_a.summary()
print("Model successfully built!")

PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "proposed_model-a"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1-a (InputLayer)          [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling-a (Rescaling)         (None, 224, 224, 3)  0           input_1-a[0][0]                  
__________________________________________________________________________________________________
normalization-a (Normalization) (None, 224, 224, 3)  7           rescaling-a[0][0]                
__________________________________________________________________________________________________
stem_conv_pad-a (ZeroPadding2D) (None, 225, 225, 3)  0           normalization-a[0][0]            
_______________________________________

In [5]:
# Builder

#TRANSFER LEARNING
def model_builder_b(model_input):
    model_builder_b = selected_model(weights='imagenet', include_top=False, input_tensor=model_input)
    
#FINE TUNING
    for layer in model_builder_b.layers:
        layer.trainable = True
    for BatchNormalization in model_builder_b.layers:
        BatchNormalization.trainable = True
    for layer in model_builder_b.layers:
        layer._name = layer.name + '-b'
        
    x = model_builder_b.layers[-192].output

    model_b = Model(inputs=model_builder_b.input, outputs=x, name=model_architecture + '-b')
    return model_b

#INITIALIZE THE MODEL
model_b = model_builder_b(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")
model_b.summary()
print("Model successfully built!")

PLEASE CHECK THE ENTIRE MODEL UP TO THE END
Model: "proposed_model-b"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1-a-b (InputLayer)        [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1-b (Rescaling)       (None, 224, 224, 3)  0           input_1-a-b[0][0]                
__________________________________________________________________________________________________
normalization_1-b (Normalizatio (None, 224, 224, 3)  7           rescaling_1-b[0][0]              
__________________________________________________________________________________________________
stem_conv_pad-b (ZeroPadding2D) (None, 225, 225, 3)  0           normalization_1-b[0][0]          
_______________________________________

<h2>Fusion of the ENEB architectures</h2>

In [6]:
#RE-INITIALIZE FOR FUSION
model_a = model_builder_a(model_input)
model_b = model_builder_b(model_input)

print("Accomplished Pre-training and ready for fusion")

Accomplished Pre-training and ready for fusion


In [7]:
#FUSE THE MODELS INTO A SINGLE PIPELINE

models = [model_a, 
          model_b]

print("Fusion success!")
print("Ready to connect with its ending layers!")

Fusion success!
Ready to connect with its ending layers!


<h2>Build the final model attached with the MReSBlock</h2>

In [10]:
#Build final model
def model_final(models, model_input):
    outputs = [m.output for m in models]
    
#Fusion
    y = Add()(outputs)
    
#Residual Layer
    y_dw1 = DepthwiseConv2D(1, 1)(y) 
    y_bn1 = BatchNormalization()(y_dw1)
    y_selu1 = activation_setting(y_bn1)
    y_conv1 = Conv2D(1, 1, kernel_initializer='lecun_normal')(y_selu1)
    
    y_dw2 = DepthwiseConv2D(1, 1)(y_conv1)
    y_bn2 = BatchNormalization()(y_dw2)
    y_selu2 = activation_setting(y_bn2)
    y_conv2 = Conv2D(1, 1, kernel_initializer='lecun_normal')(y_selu2)
    
    y_merge = Add()([y, y_conv2])
    
#FINE-TUNING
    y = GlobalAveragePooling2D()(y_merge)

    prediction = Dense(6, 
                       activation='softmax', 
                       name='softmax' + model_architecture)(y)
    
    model_final = Model(model_input, prediction, name=model_architecture)
    
    return model_final

#Istantitate the model and report the summary
model_final = model_final(models, model_input)

print()
print()
print("PLEASE CHECK THE MODEL UP TO THE END")
print()
print()
print()

print("Complete and ready for compilation and training!")
print("The error is caused because of the modifications made from the original layers")

#save the architecture
save_m(model_path + model_architecture, model_final)

#re-fresh the entire architecture into the model variable for consistency
model = load_model(model_path + model_architecture + '/' + model_architecture + '.h5')

model.summary()



PLEASE CHECK THE MODEL UP TO THE END



Complete and ready for compilation and training!
The error is caused because of the modifications made from the original layers
model saved
Model: "proposed_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1-a-b-a-b (InputLayer)    [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2-a (Rescaling)       (None, 224, 224, 3)  0           input_1-a-b-a-b[0][0]            
__________________________________________________________________________________________________
rescaling_3-b (Rescaling)       (None, 224, 224, 3)  0           input_1-a-b-a-b[0][0]            
__________________________________________________________________________________________________
no